## Get all plugins json
1. Open devtools
2. Navigate to ChatGPT plugins
3. Filter by `https://chat.openai.com/backend-api/aip/p?offset=0`
4. Click `p?offset=0...` then click on tab `Preview`
5. Copy the value of `items` to `all_plugins.json`

In [9]:
# DATA FILE

import json
import os
import re
import requests

pypi_path = 'all_plugins.json'

with open(pypi_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Concatenate lines into a single JSON document
json_content = ''.join(lines)

try:
    data = json.loads(json_content)
except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")

In [10]:
# get the json from blacklist.json
with open('blacklist.json', 'r', encoding='utf-8') as f:
    blacklist = json.load(f)
def aggregate_items(json_object):
    aggregate_list = []
    for key in json_object:
        aggregate_list.extend(json_object[key])
    return aggregate_list
blacklist_list = aggregate_items(blacklist)
blacklist_list

['Coursera', 'video_insights', 'KAYAK', 'staypia', 'portfoliopilot']

In [11]:
# ACCEPTED PLUGINS

with open('blacklist.json', 'r', encoding='utf-8') as f:
    blacklist = json.load(f)
def aggregate_items(json_object):
    aggregate_list = []
    for key in json_object:
        aggregate_list.extend(json_object[key])
    return aggregate_list
blacklist_list = aggregate_items(blacklist)

accepted_plugins = []
data = sorted(data, key=lambda k: k['namespace'])
for idx, plugin in enumerate(data):
    if idx % 20 == 0:
        print(f"Checking plugin {idx} out of {len(data)}")
    if plugin and plugin.get("manifest", {}).get("auth", {}).get("type") == "none" and plugin.get("namespace") not in blacklist_list:
        name = plugin.get("namespace")
        # ping the manifest.api.url to see if it is accessible and if so, add it to the plugins.json
        openapi_url = plugin.get("manifest", {}).get("api", {}).get("url")
        if openapi_url:
            try:
                response = requests.get(openapi_url)
                if response.status_code == 200:
                    accepted_plugins.append(plugin)
                else:
                    print(f"Could not access '{name}' with status code {response.status_code}")
            except Exception as e:
                print(f"Could not access '{name}' with error: {e}")
print(f"Accepted plugins {len(accepted_plugins)} out of {len(data)}")

Checking plugin 0 out of 548
Could not access 'AINewsRoundup' with status code 406
Could not access 'AusPetrolPrices' with status code 406
Could not access 'AusSurfReport' with status code 406
Checking plugin 20 out of 548
Could not access 'BitcoinSentiment' with status code 406
Could not access 'CSVExport' with status code 403
Could not access 'ChatWithGit' with status code 503
Checking plugin 40 out of 548
Could not access 'ContentRewriter' with status code 406
Checking plugin 60 out of 548
Could not access 'FinTorch_Trading_Assistant' with error: Invalid URL '/openapi.yaml': No scheme supplied. Perhaps you meant https:///openapi.yaml?
Could not access 'Free_Kiddie_Books' with error: Invalid URL '/openapi.json': No scheme supplied. Perhaps you meant https:///openapi.json?
Could not access 'GeoGuru' with error: Invalid URL '/openapi.json': No scheme supplied. Perhaps you meant https:///openapi.json?
Could not access 'Glowing' with status code 404
Could not access 'ImageEditor' with er

In [12]:
# CREATE plugins.json

# for each plugin
all_plugins = {"__testing__": "http://localhost:3333"}
for plugin in accepted_plugins:
    name = plugin.get("namespace")
    all_plugins[name] = "https://" + plugin.get("domain")

# Navigate two directories back from the current working directory
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir))

pypi_path = os.path.join(root_dir, 'pypi-client', 'openpluginclient', 'plugins.json')
with open(pypi_path, 'w') as f:
    json.dump(all_plugins, f, indent=4, sort_keys=True)
# npm_path = os.path.join(parent_dir, 'npm-client', ..., 'plugins.json')
# with open(npm_path, 'w') as f:
#     json.dump(all_plugins, f, indent=4, sort_keys=True)

print(f"Successfully created plugins.json files with {len(all_plugins)} plugins.")

Successfully created plugins.json files with 341 plugins.


In [13]:
# CREATE PLUGINS.md

# Create plugins file
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir))

# Create the file path for plugins.md
pluginsmd_path = os.path.join(root_dir, 'PLUGINS.md')
plugins_md = """
# Plugins
Available plugins for OpenPlugin

| Image | Namespace | Description | Description for model |
| --- | --- | --- | --- |
"""

def escape_special_markdown_chars(text):
    # Characters to escape: \ ` * _ { } [ ] ( ) # + !
    special_chars = r'\\|`|\*|_|{|}|\[|\]|\(|\)|#|\+|!'
    return re.sub(special_chars, lambda match: '\\' + match.group(), text)

def remove_line_breaks(text):
    return text.replace('\n', ' ').replace('\r', '')

for idx, plugin in enumerate(accepted_plugins):
        if idx % 20 == 0:
            print(f"Checking plugin {idx} out of {len(accepted_plugins)}")
        openapi_url = plugin.get("manifest", {}).get("api", {}).get("url")
        if openapi_url:
            try:
                # TODO: this request is repetitive and it could be handled at the ACCEPTED PLUGINS stage
                response = requests.get(openapi_url)
                if response.status_code == 200:
                    image = escape_special_markdown_chars(plugin.get("manifest", {}).get("logo_url"))
                    namespace = escape_special_markdown_chars(plugin.get("namespace"))
                    description = escape_special_markdown_chars(remove_line_breaks(plugin.get("manifest", {}).get("description_for_human")))
                    description_for_model = escape_special_markdown_chars(remove_line_breaks(plugin.get("manifest", {}).get("description_for_model")))
                    plugins_md += f"| ![{namespace} Logo]({image}) | {namespace} | {description} | {description_for_model} |\n"
            except:
                print(f"Error with {plugin.get('namespace')}")

with open(pluginsmd_path, 'w', encoding='utf-8') as f:
    f.write(plugins_md)

print(f"Successfully created PLUGINS.md file with {len(accepted_plugins)} plugins of {len(data)} total plugins.")


Checking plugin 0 out of 342
Checking plugin 20 out of 342
Checking plugin 40 out of 342
Checking plugin 60 out of 342
Checking plugin 80 out of 342
Checking plugin 100 out of 342
Checking plugin 120 out of 342
Checking plugin 140 out of 342
Checking plugin 160 out of 342
Checking plugin 180 out of 342
Checking plugin 200 out of 342
Checking plugin 220 out of 342
Checking plugin 240 out of 342
Checking plugin 260 out of 342
Checking plugin 280 out of 342
Checking plugin 300 out of 342
Checking plugin 320 out of 342
Checking plugin 340 out of 342
Successfully created PLUGINS.md file with 342 plugins of 548 total plugins.
